In [1]:
import pandas as pd
import ast
import pytz
import os
import json
import yaml
import glob
from dateutil import tz
from datetime import datetime

class BasePipeline:
    def __init__(self, config, base_file_path):
        self.config = config
        self.base_file_path = base_file_path
        self.geomap = {}
        self._data = pd.DataFrame({})
    
    def construct_file_path(self):
        #TODO: add type to prefix mapping
        file_prefix = 'per_outage' if self.config['type'] == 'o' else 'per_county'
        file_path = f"{self.base_file_path}/{self.config['state']}/layout_{self.config['layout']}/{file_prefix}_{self.config['name']}.csv"
        return file_path.replace('//', '/')

    def load_data(self):
        try:
            file_path = self.construct_file_path()
            print(file_path)
            self._data = pd.read_csv(file_path)
            with open('zip_to_county_name.json', 'r') as json_file:
                self.geomap['zip_to_county_name'] = json.load(json_file)
            with open('zip_to_county_fips.json', 'r') as json_file:
                self.geomap['zip_to_county_fips'] = json.load(json_file)
        except Exception as e:
            print(f"An error occurred during file loading: {e}")
            
    def transform(self):
        raise NotImplementedError

    def standardize(self):
        """
        Generic method to compute and output standardized metrics
        """
        self.load_data()
        self.transform()
        grouped = self._data.groupby('outage_id').apply(self._compute_metrics).reset_index().round(2)
        self._data = pd.merge(grouped, self._data, on=['outage_id', 'timestamp'], how='inner')
        
        self._data['state'] = self.config['state']
        if self.config['state'] != 'ca':
            self._data['utility_provider'] = self.config['name'] 
            self._data['county'] = self._data['zipcode'].map(self.geomap) 
        
        self._data = self._data[[
            'utility_provider', 'state', 'county', 'zipcode',
            'outage_id', 'start_time', 'end_time', 'lat', 'lng', 
            'duration', 'duration_max', 'duration_mean', 'customer_affected_mean', 'total_customer_outage_time', 'total_customer_outage_time_max', 'total_customer_outage_time_mean'
        ]]
        
        return self._data
    
    def output_data(self, standard_data):
        # TODO: Output unified data
        pass
    
    def get_dataframe(self):
        return self._data
    
    def _compute_metrics(self, group):
        """
        Generic method to compute standardized metrics, used for being apply in DataFrame.groupby method, 
        given dataframe being transformed with standardized column names
        """
        duration = (group['end_time'] - group['start_time']).dt.total_seconds() / 60
        duration_max = duration + 15
        duration_mean = (duration + duration_max) / 2
        customer_affected_mean = group['customer_affected'].mean()
        
        total_customer_outage_time = 15 * (group['customer_affected'].sum() - group['customer_affected'].iloc[0]) + (group['timestamp'].iloc[0] - group['start_time'].iloc[0]).total_seconds() / 60 * group['customer_affected'].iloc[0]
        total_customer_outage_time_max = total_customer_outage_time + 15 * group['customer_affected'].iloc[-1]
        total_customer_outage_time_mean = (total_customer_outage_time + total_customer_outage_time_max) / 2

        return pd.Series({
            'timestamp': group['end_time'].iloc[-1],
            'duration': duration.iloc[-1],
            'duration_max': duration_max.iloc[-1],
            'duration_mean': duration_mean.iloc[-1],
            'customer_affected_mean': customer_affected_mean,
            'total_customer_outage_time': total_customer_outage_time,
            'total_customer_outage_time_max': total_customer_outage_time_max,
            'total_customer_outage_time_mean': total_customer_outage_time_mean
        })
        
    def check_vars(self):
        # TODO: Check other useful variables
        pass

In [14]:
class GA3TX16(BasePipeline):
    def transform(self):
        try:
            # Convert timestamps
            eastern = tz.gettz('US/Eastern')
            self._data['timestamp'] = pd.to_datetime(self._data['timestamp'], utc=True).dt.tz_convert(eastern)
            self._data['OutageTime'] = pd.to_datetime(self._data['OutageTime'], utc=True).dt.tz_convert(eastern)
            self._data['end_time'] = self._data.groupby('CaseNumber')['timestamp'].transform('max')
            
             # TODO: get zipcode from lat and long
            self._data['zipcode'] = "000000"
            
            self._data = self._data.rename(columns={
                'CaseNumber':'outage_id',
                'OutageTime': 'start_time',
                'CutomersAffected':'customer_affected',
                'X':'lat',
                'Y': 'lng'
            })
        except Exception as e:
            print(f"An error occurred during transformation: {e}")

In [16]:
from IPython.display import display
pd.set_option('display.max_columns', None)

with open('/Users/xuanedx1/github/outage-data-scraper/app/pipeline/config.yaml', 'r') as file:
    config = yaml.safe_load(file)
    base_file_path = config['globals']['LOCAL_FILE_BASE_PATH']

# Instantiate a BasePipeline object for each provider in the configuration
for provider in config['providers']:
    pipeline = GA3TX16(provider, base_file_path)
    pipeline.standardize()
    display(pipeline._data)
    # pipeline._data.to_csv(f"{pipeline.config['name']}_o.csv")

/Users/xuanedx1/github/outage-data-scraper/data/s3/ga/layout_3/per_outage_Coastal Electric Cooperation.csv


,utility_provider,state,county,zipcode,outage_id,start_time,end_time,lat,lng,duration,duration_max,duration_mean,customer_affected_mean,total_customer_outage_time,total_customer_outage_time_max,total_customer_outage_time_mean
0,Coastal Electric Cooperation,ga,NaN,000000,231011-P0003,2023-10-11 18:53:00-04:00,2023-10-12 00:26:43-04:00,-81.276932,31.701333,333.72,348.72,341.22,40.0,12149.33,12749.33,12449.33
1,Coastal Electric Cooperation,ga,NaN,000000,231011-X0003,2023-10-11 18:53:00-04:00,2023-10-11 22:56:43-04:00,-81.289966,31.694120,243.72,258.72,251.22,2.0,487.43,517.43,502.43
2,Coastal Electric Cooperation,ga,NaN,000000,231012-P0003,2023-10-12 05:25:00-04:00,2023-10-12 10:26:45-04:00,-81.299617,31.753929,301.75,316.75,309.25,26.0,7845.93,8235.93,8040.93
3,Coastal Electric Cooperation,ga,NaN,000000,231012-P0006,2023-10-12 05:43:00-04:00,2023-10-12 11:26:45-04:00,-81.273516,31.873195,343.75,358.75,351.25,1.0,343.75,358.75,351.25
4,Coastal Electric Cooperation,ga,NaN,000000,231012-P0011,2023-10-12 06:06:00-04:00,2023-10-12 10:41:46-04:00,-81.451307,31.768307,275.77,290.77,283.27,94.0,25920.50,27330.50,26625.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,Coastal Electric Cooperation,ga,NaN,000000,240130-X0019,2024-01-30 09:47:00-05:00,2024-01-30 14:56:47-05:00,-81.384942,31.646037,309.78,324.78,317.28,2.0,619.57,649.57,634.57
479,Coastal Electric Cooperation,ga,NaN,000000,240131-P0002,2024-01-31 03:56:00-05:00,2024-01-31 09:11:48-05:00,-81.470093,31.799693,315.80,330.80,323.30,2.0,631.60,661.60,646.60
480,Coastal Electric Cooperation,ga,NaN,000000,240131-P0006,2024-01-31 06:37:00-05:00,2024-01-31 11:56:48-05:00,-81.606561,31.764091,319.80,334.80,327.30,13.0,4157.40,4352.40,4254.90
481,Coastal Electric Cooperation,ga,NaN,000000,240131-P0010,2024-01-31 07:06:00-05:00,2024-01-31 12:26:49-05:00,-81.626020,31.755771,320.82,335.82,328.32,14.0,4491.43,4701.43,4596.43


/Users/xuanedx1/github/outage-data-scraper/data/s3/ga/layout_3/per_outage_Irwin EMC.csv


,utility_provider,state,county,zipcode,outage_id,start_time,end_time,lat,lng,duration,duration_max,duration_mean,customer_affected_mean,total_customer_outage_time,total_customer_outage_time_max,total_customer_outage_time_mean
0,Irwin EMC,ga,NaN,000000,230413-O0001,2023-04-13 07:24:00-04:00,2023-04-13 12:56:42-04:00,-83.622262,31.737536,332.70,347.70,340.20,1.0,332.70,347.70,340.20
1,Irwin EMC,ga,NaN,000000,230413-P0003,2023-04-13 09:22:00-04:00,2023-04-13 13:56:42-04:00,-83.109390,31.664980,274.70,289.70,282.20,4.0,609.40,699.40,654.40
2,Irwin EMC,ga,NaN,000000,230413-X0003,2023-04-13 09:22:00-04:00,2023-04-13 13:26:43-04:00,-83.109395,31.664873,244.72,259.72,252.22,2.0,489.43,519.43,504.43
3,Irwin EMC,ga,NaN,000000,230414-P0001,2023-04-14 01:44:00-04:00,2023-04-14 06:56:42-04:00,-83.054233,31.624676,312.70,327.70,320.20,1.0,312.70,327.70,320.20
4,Irwin EMC,ga,NaN,000000,230414-P0003,2023-04-14 16:51:00-04:00,2023-04-14 21:41:43-04:00,-83.156380,31.740883,290.72,305.72,298.22,1.0,290.70,305.70,298.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2654,Irwin EMC,ga,NaN,000000,240202-P0004,2024-02-02 04:39:18-05:00,2024-02-02 11:41:43-05:00,-83.570311,31.569982,422.42,437.42,429.92,51.0,21542.40,22307.40,21924.90
2655,Irwin EMC,ga,NaN,000000,240202-P0007,2024-02-02 07:42:00-05:00,2024-02-02 16:56:42-05:00,-83.709955,31.660589,554.70,569.70,562.20,1.0,554.70,569.70,562.20
2656,Irwin EMC,ga,NaN,000000,240202-X0001,2024-02-02 04:39:18-05:00,2024-02-02 09:41:42-05:00,-83.595335,31.574544,302.40,317.40,309.90,1.0,302.40,317.40,309.90
2657,Irwin EMC,ga,NaN,000000,240202-X0004,2024-02-02 04:39:18-05:00,2024-02-02 09:56:42-05:00,-83.570311,31.569982,317.40,332.40,324.90,39.0,12378.60,12963.60,12671.10


/Users/xuanedx1/github/outage-data-scraper/data/s3/ga/layout_3/per_outage_Middle Georgia EMC.csv


...

,utility_provider,state,county,zipcode,outage_id,start_time,end_time,lat,lng,duration,duration_max,duration_mean,customer_affected_mean,total_customer_outage_time,total_customer_outage_time_max,total_customer_outage_time_mean
0,Upson EMC,ga,NaN,000000,230414-O0016,2023-04-14 08:20:00-04:00,2023-04-14 13:11:44-04:00,-84.471960,32.691098,291.73,306.73,299.23,206.0,60097.07,63187.07,61642.07
1,Upson EMC,ga,NaN,000000,230414-P0005,2023-04-14 02:58:00-04:00,2023-04-14 08:41:44-04:00,-84.296393,32.949948,343.73,358.73,351.23,28.0,9624.53,10044.53,9834.53
2,Upson EMC,ga,NaN,000000,230414-P0006,2023-04-14 02:59:00-04:00,2023-04-14 08:41:44-04:00,-84.311164,32.899785,342.73,357.73,350.23,116.0,39757.07,41497.07,40627.07
3,Upson EMC,ga,NaN,000000,230414-P0009,2023-04-14 03:08:00-04:00,2023-04-14 08:41:44-04:00,-84.305402,32.910190,333.73,348.73,341.23,1.0,333.73,348.73,341.23
4,Upson EMC,ga,NaN,000000,230414-P0012,2023-04-14 04:55:00-04:00,2023-04-14 09:41:44-04:00,-84.387516,33.030568,286.73,301.73,294.23,23.0,6594.87,6939.87,6767.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,Upson EMC,ga,NaN,000000,240127-P0023,2024-01-27 13:29:00-05:00,2024-01-27 18:56:48-05:00,-84.419305,32.946959,327.80,342.80,335.30,2.0,655.60,685.60,670.60
1124,Upson EMC,ga,NaN,000000,240127-P0025,2024-01-27 14:41:00-05:00,2024-01-27 20:11:48-05:00,-84.422117,32.947752,330.80,345.80,338.30,1.0,330.80,345.80,338.30
1125,Upson EMC,ga,NaN,000000,240127-X0003,2024-01-27 04:52:00-05:00,2024-01-27 09:56:58-05:00,-84.474079,32.916434,304.97,319.97,312.47,1.0,304.97,319.97,312.47
1126,Upson EMC,ga,NaN,000000,240127-X0004,2024-01-27 04:55:00-05:00,2024-01-27 09:56:58-05:00,-84.432160,32.955757,301.97,316.97,309.47,2.0,603.93,633.93,618.93


/Users/xuanedx1/github/outage-data-scraper/data/s3/tx/layout_16/per_outage_Comanche Electric Coop.csv


,utility_provider,state,county,zipcode,outage_id,start_time,end_time,lat,lng,duration,duration_max,duration_mean,customer_affected_mean,total_customer_outage_time,total_customer_outage_time_max,total_customer_outage_time_mean
0,Comanche Electric Coop,tx,NaN,000000,230406-X0001,2023-04-06 04:44:59-04:00,2023-04-06 09:47:26-04:00,-99.090857,32.185723,302.45,317.45,309.95,1.0,302.45,317.45,309.95
1,Comanche Electric Coop,tx,NaN,000000,230406-X0008,2023-04-06 08:59:38-04:00,2023-04-06 14:02:26-04:00,-98.788941,32.110586,302.80,317.80,310.30,1.0,302.80,317.80,310.30
2,Comanche Electric Coop,tx,NaN,000000,230407-O0013,2023-04-07 15:30:36-04:00,2023-04-07 21:02:26-04:00,-98.829156,32.569059,331.83,346.83,339.33,218.0,72339.67,75609.67,73974.67
3,Comanche Electric Coop,tx,NaN,000000,230407-P0001,2023-04-07 08:25:23-04:00,2023-04-07 14:02:26-04:00,-98.561390,31.960670,337.05,352.05,344.55,1.0,337.05,352.05,344.55
4,Comanche Electric Coop,tx,NaN,000000,230407-X0001,2023-04-07 08:25:23-04:00,2023-04-07 13:32:27-04:00,-98.561390,31.960670,307.07,322.07,314.57,1.0,307.07,322.07,314.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124,Comanche Electric Coop,tx,NaN,000000,240202-X0022,2024-02-02 14:54:16-05:00,2024-02-02 21:02:26-05:00,-99.146133,32.386604,368.17,383.17,375.67,6.0,2209.00,2299.00,2254.00
3125,Comanche Electric Coop,tx,NaN,000000,240202-X0031,2024-02-02 15:43:29-05:00,2024-02-02 21:47:26-05:00,-98.741360,32.259905,363.95,378.95,371.45,1.0,363.95,378.95,371.45
3126,Comanche Electric Coop,tx,NaN,000000,240202-X0034,2024-02-02 15:54:10-05:00,2024-02-02 22:17:26-05:00,-98.729032,32.099213,383.27,398.27,390.77,11.0,4215.93,4380.93,4298.43
3127,Comanche Electric Coop,tx,NaN,000000,240202-X0036,2024-02-02 15:56:36-05:00,2024-02-02 22:02:26-05:00,-98.631310,31.801981,365.83,380.83,373.33,26.0,9511.67,9901.67,9706.67


/Users/xuanedx1/github/outage-data-scraper/data/s3/tx/layout_16/per_outage_Southwestern Electric Coop, Inc..csv


,utility_provider,state,county,zipcode,outage_id,start_time,end_time,lat,lng,duration,duration_max,duration_mean,customer_affected_mean,total_customer_outage_time,total_customer_outage_time_max,total_customer_outage_time_mean
0,"Southwestern Electric Coop, Inc.",tx,NaN,000000,230405-P0063,2023-04-05 17:57:01-04:00,2023-04-06 02:17:26-04:00,-89.846474,38.953885,500.42,515.42,507.92,43.0,21428.33,22073.33,21750.83
1,"Southwestern Electric Coop, Inc.",tx,NaN,000000,230406-O0008,2023-04-06 05:40:00-04:00,2023-04-06 12:17:26-04:00,-88.838475,39.229915,397.43,412.43,404.93,1.0,397.43,412.43,404.93
2,"Southwestern Electric Coop, Inc.",tx,NaN,000000,230406-O0013,2023-04-06 12:17:00-04:00,2023-04-06 17:32:26-04:00,-89.212782,38.998155,315.43,330.43,322.93,25.0,7885.83,8260.83,8073.33
3,"Southwestern Electric Coop, Inc.",tx,NaN,000000,230406-P0001,2023-04-06 04:50:09-04:00,2023-04-06 10:02:26-04:00,-89.978226,38.725058,312.28,327.28,319.78,2.0,624.57,654.57,639.57
4,"Southwestern Electric Coop, Inc.",tx,NaN,000000,230406-P0013,2023-04-06 11:44:59-04:00,2023-04-06 17:17:26-04:00,-89.212782,38.998155,332.45,347.45,339.95,25.0,8311.25,8686.25,8498.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3452,"Southwestern Electric Coop, Inc.",tx,NaN,000000,240202-O0001,2024-02-02 02:20:54-05:00,2024-02-02 10:32:26-05:00,-88.711030,39.002838,491.53,506.53,499.03,40.0,19660.67,20260.67,19960.67
3453,"Southwestern Electric Coop, Inc.",tx,NaN,000000,240202-O0002,2024-02-02 03:32:50-05:00,2024-02-02 10:17:27-05:00,-89.600226,38.816203,404.62,419.62,412.12,1.0,404.60,419.60,412.10
3454,"Southwestern Electric Coop, Inc.",tx,NaN,000000,240202-O0004,2024-02-02 08:00:00-05:00,2024-02-02 14:47:25-05:00,-89.918627,38.742225,407.42,422.42,414.92,1.0,407.43,422.43,414.93
3455,"Southwestern Electric Coop, Inc.",tx,NaN,000000,240202-P0001,2024-02-02 02:20:54-05:00,2024-02-02 09:47:26-05:00,-88.711030,39.002838,446.53,461.53,454.03,40.0,17860.67,18460.67,18160.67


---

In [6]:
df = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/s3/ga/layout_3/per_outage_Coastal Electric Cooperation.csv')
df

,OutageTime,CaseNumber,CutomersAffected,PoleNumber,RestorationTime,StartTime,EndTime,ElementName,CaseStatus,Cause,X,Y,timestamp,EMC
0,10/11/2023 10:53:00 PM,231011-X0003,2,743 CATTLE HAMMOCK RD,True,0001-01-01T00:00:00,0001-01-01T00:00:00,NaN,Calls Bundle,NaN,-81.289966,31.694120,10-12-2023 02:56:43,Coastal Electric Cooperation
1,10/11/2023 10:53:00 PM,231011-P0003,40,CAMELLIA RD COLONELS ISLAND,True,0001-01-01T00:00:00,0001-01-01T00:00:00,R3450022,Predicted,NaN,-81.276932,31.701333,10-12-2023 03:11:44,Coastal Electric Cooperation
2,10/11/2023 10:53:00 PM,231011-P0003,40,CAMELLIA RD COLONELS ISLAND,True,0001-01-01T00:00:00,0001-01-01T00:00:00,R3450022,Predicted,NaN,-81.276932,31.701333,10-12-2023 03:26:43,Coastal Electric Cooperation
3,10/11/2023 10:53:00 PM,231011-P0003,40,CAMELLIA RD COLONELS ISLAND,True,0001-01-01T00:00:00,0001-01-01T00:00:00,R3450022,Predicted,NaN,-81.276932,31.701333,10-12-2023 03:56:44,Coastal Electric Cooperation
4,10/11/2023 10:53:00 PM,231011-P0003,40,CAMELLIA RD COLONELS ISLAND,True,0001-01-01T00:00:00,0001-01-01T00:00:00,R3450022,Predicted,NaN,-81.276932,31.701333,10-12-2023 04:26:43,Coastal Electric Cooperation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586,1/31/2024 8:56:00 AM,240131-P0002,2,229 ROBINSON CIR,true,0001-01-01T00:00:00,0001-01-01T00:00:00,T2930157,Predicted,NaN,-81.470093,31.799693,01-31-2024 14:11:48,Coastal Electric Cooperation
1587,1/31/2024 11:37:00 AM,240131-P0006,13,37 GLENBROOK,true,0001-01-01T00:00:00,0001-01-01T00:00:00,T2910538,Predicted,NaN,-81.606561,31.764091,01-31-2024 16:56:48,Coastal Electric Cooperation
1588,1/31/2024 12:06:00 PM,240131-P0010,14,SHERWOOD FORREST GRIFFIN,true,0001-01-01T00:00:00,0001-01-01T00:00:00,F2910047,Predicted,NaN,-81.626020,31.755771,01-31-2024 17:26:49,Coastal Electric Cooperation
1589,1/31/2024 12:54:00 PM,240131-P0012,29,OAK RIDGE MHP OFF TIBET,true,0001-01-01T00:00:00,0001-01-01T00:00:00,F3410002,Predicted,NaN,-81.615985,31.747205,01-31-2024 17:56:48,Coastal Electric Cooperation


In [7]:
df[df['CaseNumber'] == '231011-X0003']

,OutageTime,CaseNumber,CutomersAffected,PoleNumber,RestorationTime,StartTime,EndTime,ElementName,CaseStatus,Cause,X,Y,timestamp,EMC
0,10/11/2023 10:53:00 PM,231011-X0003,2,743 CATTLE HAMMOCK RD,True,0001-01-01T00:00:00,0001-01-01T00:00:00,NaN,Calls Bundle,NaN,-81.289966,31.69412,10-12-2023 02:56:43,Coastal Electric Cooperation


In [9]:
df[df['CaseNumber'] == '231011-P0003']

,OutageTime,CaseNumber,CutomersAffected,PoleNumber,RestorationTime,StartTime,EndTime,ElementName,CaseStatus,Cause,X,Y,timestamp,EMC
1,10/11/2023 10:53:00 PM,231011-P0003,40,CAMELLIA RD COLONELS ISLAND,True,0001-01-01T00:00:00,0001-01-01T00:00:00,R3450022,Predicted,NaN,-81.276932,31.701333,10-12-2023 03:11:44,Coastal Electric Cooperation
2,10/11/2023 10:53:00 PM,231011-P0003,40,CAMELLIA RD COLONELS ISLAND,True,0001-01-01T00:00:00,0001-01-01T00:00:00,R3450022,Predicted,NaN,-81.276932,31.701333,10-12-2023 03:26:43,Coastal Electric Cooperation
3,10/11/2023 10:53:00 PM,231011-P0003,40,CAMELLIA RD COLONELS ISLAND,True,0001-01-01T00:00:00,0001-01-01T00:00:00,R3450022,Predicted,NaN,-81.276932,31.701333,10-12-2023 03:56:44,Coastal Electric Cooperation
4,10/11/2023 10:53:00 PM,231011-P0003,40,CAMELLIA RD COLONELS ISLAND,True,0001-01-01T00:00:00,0001-01-01T00:00:00,R3450022,Predicted,NaN,-81.276932,31.701333,10-12-2023 04:26:43,Coastal Electric Cooperation


In [10]:
df[df['CaseNumber'] == '240131-P0012']

,OutageTime,CaseNumber,CutomersAffected,PoleNumber,RestorationTime,StartTime,EndTime,ElementName,CaseStatus,Cause,X,Y,timestamp,EMC
1589,1/31/2024 12:54:00 PM,240131-P0012,29,OAK RIDGE MHP OFF TIBET,true,0001-01-01T00:00:00,0001-01-01T00:00:00,F3410002,Predicted,NaN,-81.615985,31.747205,01-31-2024 17:56:48,Coastal Electric Cooperation
1590,1/31/2024 12:54:00 PM,240131-P0012,29,OAK RIDGE MHP OFF TIBET,true,0001-01-01T00:00:00,0001-01-01T00:00:00,F3410002,Predicted,NaN,-81.615985,31.747205,01-31-2024 18:11:47,Coastal Electric Cooperation


In [11]:
df[df['CaseNumber'] == '240131-P0010']

,OutageTime,CaseNumber,CutomersAffected,PoleNumber,RestorationTime,StartTime,EndTime,ElementName,CaseStatus,Cause,X,Y,timestamp,EMC
1588,1/31/2024 12:06:00 PM,240131-P0010,14,SHERWOOD FORREST GRIFFIN,true,0001-01-01T00:00:00,0001-01-01T00:00:00,F2910047,Predicted,NaN,-81.62602,31.755771,01-31-2024 17:26:49,Coastal Electric Cooperation
